In [1]:
import pandas as pd

Train=pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/train.csv')
Test= pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/test.csv')
sample_submission= pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/sample_submission.csv')

In [2]:
#replace ".."with .100
# new_value = .210
new_value = 0.100
Train=Train.replace('..', new_value)  
Test=Test.replace('..', new_value)

In [3]:
#Unique country for iteration
unique = Train['Country Name'].unique()

In [4]:
#trainig and inferencing with LogisticRegression and ARIMA
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
# Ignore ValueWarning issued by statsmodels
# warnings.filterwarnings("ignore", category=ValueWarning)
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression, HuberRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, HistGradientBoostingRegressor
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
j=[]
for i in unique:
    transposed_df=0
    fg = Train[Train['Country Name'] == i].reset_index(drop=True)
    columns_to_drop = ['Country Code', 'Country Name','Indicator']
    fg = fg.drop(columns_to_drop, axis=1)
    transposed_df = fg.transpose()
    #print(fg)
    #print('done for now')
    Xtrain = transposed_df[[0,1, 2,3,4,5,6,7,8,9,10]]
    Xtrain
    Ytrain=transposed_df[11]
    Ytrain
    # logistic_regression = LogisticRegression()
    # Initialize models
    lr = LogisticRegression()
    lnr = LinearRegression()
    dt = DecisionTreeRegressor(max_depth=4, random_state=42)
    rf = RandomForestRegressor(max_depth=4, random_state=42)
    hgb = HistGradientBoostingRegressor(random_state=42)
    et = ExtraTreesRegressor(n_estimators=100, random_state=42)
    hb = HuberRegressor()
    
    # Create the voting ensemble
    voting_regressor = VotingRegressor([
        ('lr', lr),
        ('lnr', lnr), 
        ('dt', dt), 
        ('rf', rf), 
        ('hgb', hgb), 
        ('et', et), 
        ('hb', hb)
    ])
    
    try:
        #logistic_regression.fit(Xtrain,Ytrain)
        voting_regressor.fit(Xtrain, Ytrain)
        fgt = Test[Test['Country Name'] == i].reset_index(drop=True)
        columns_to_drop = [ 'Country Name','Indicator']
        fgt = fgt.drop(columns_to_drop, axis=1)
        transposed_df = fgt.transpose()
        #print(transposed_df)
  
        Xtest = transposed_df[[0,1, 2,3,4,5,6,7,8,9,10]]
        # forecast = logistic_regression.predict(Xtest.astype(float))
        forecast = voting_regressor.predict(Xtest.astype(float))
        
        model = ARIMA(Ytrain.astype(float), order=(0,1,0))  # Adjust order as needed
        model_fit = model.fit()

    
        forecast2 = model_fit.forecast(steps=16)
        
        row_index = sample_submission[sample_submission.eq(i).any(axis=1)].index[0]


        new_values = [i, forecast[0], forecast[1],forecast[2], forecast[3], forecast[4],forecast2[30]]
        sample_submission.loc[row_index] = new_values
    except:
        j.append(i)

In [ ]:
#trainig and inferencing with LogisticRegression and ARIMA
import warnings
import tensorflow as tf
warnings.filterwarnings("ignore")
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Dropout


k=0
for i in unique:
    transposed_df=0
    fg = Train[Train['Country Name'] == i].reset_index(drop=True)
    columns_to_drop = ['Country Code', 'Country Name','Indicator']
    fg = fg.drop(columns_to_drop, axis=1)
    transposed_df = fg.transpose()
    #print(fg)
    #print('done for now')
    Xtrain = transposed_df[[0,1, 2,3,4,9,10]].astype(float)
    xt=Xtrain.values
    Ytrain=transposed_df[11].astype(float) 
    yt=Ytrain.values
    
    if k == 0:
        k+=1
        model = Sequential([
              LSTM(64, return_sequences=True, input_shape=(Xtrain.shape[1], 1)),
              Dropout(0.2),  # Adding dropout for regularization
              LSTM(64, return_sequences=True),
              Dropout(0.2),
              LSTM(64),  # Last LSTM layer without return sequences
              Dropout(0.2),
              Dense(32, activation='relu'),  # Adding a dense layer for additional processing
              Dense(1)  # Output layer
              ])
        model.compile(optimizer = tf.keras.optimizers.Adam(), loss=tf.keras.losses.MeanSquaredError())
        Xtrain = xt.reshape((xt.shape[0], xt.shape[1], 1))
        # model.fit( Xtrain,  yt, epochs=2, batch_size=2)
        # model.fit( Xtrain,  yt, epochs=100, batch_size=32)
        model.fit( Xtrain,  yt, epochs=50, batch_size=16)
        model.save("/kaggle/working/my_lstm_model.h5")
    else:
        loaded_model = tf.keras.models.load_model("/kaggle/working/my_lstm_model.h5")
        loaded_model.compile(optimizer = tf.keras.optimizers.Adam(), loss=tf.keras.losses.MeanSquaredError())
        # loaded_model.fit(Xtrain, yt, epochs=2, batch_size=2)
        # loaded_model.fit(Xtrain, yt, epochs=100, batch_size=32)
        loaded_model.fit(Xtrain, yt, epochs=50, batch_size=16)
        loaded_model.save("/kaggle/working/my_lstm_model.h5")

2024-04-03 09:06:19.498599: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-03 09:06:19.498691: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-03 09:06:19.659443: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - loss: 0.0906
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0386
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0228
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0176
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0306
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0255
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0187
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0184
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0135
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0131
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0210
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0174
Epoch 13/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0164
Epoch 14/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0137
Epoch 15/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0131
Epoch 16/50
1/1 ━━━━━━━━━━━━━━━━━━━━

In [ ]:
#predicting and forecasting unknown Co2 Value
for i in j:
    fg = Test[Test['Country Name'] == i].reset_index(drop=True)
    columns_to_drop = ['Country Name','Indicator']
    fg = fg.drop(columns_to_drop, axis=1)
    transposed_df = fg.transpose()
    #print(fg)
    #print('done for now')
    XTest = transposed_df[[0,1, 2,3,4,9,10]]
    loaded_model = tf.keras.models.load_model("/kaggle/working/my_lstm_model.h5")
   
    forecast = loaded_model.predict(XTest.astype(float))
    df = pd.DataFrame(forecast)
    model2 = ARIMA(df[0], order=(0,1,0))  # Adjust order as needed
    model_fit = model2.fit()

    
    forecast2 = model_fit.forecast(steps=16)
        
    row_index = sample_submission[sample_submission.eq(i).any(axis=1)].index[0]


    new_values = [i, forecast[0][0], forecast[1][0],forecast[2][0], forecast[3][0], forecast[4][0],forecast2[14]]
    sample_submission.loc[row_index] = new_values

In [ ]:
sample_submission.head(15)

In [ ]:
sample_submission.to_csv('submission_03_04_2024_03_PM.csv', index=False)